In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import Dataset
from scipy.interpolate import interp1d
import auxiliary_lib as au
import isamcalc_lib as isam
import subprocess
import socplot_lib as socplt


In [2]:
#======================================================
## Organize the data
#======================================================
totlen = 403248     # From 1995 to 2017
totyrrange = 1995 + np.arange(23)
day_of_year = [365, 366, 365, 365, 365, 366, 365, 365, 365, 366, 365, \
               365, 365, 366, 365, 365, 365, 366, 365, 365, 365, 366, 365]   # DOY from 1995 to 2017
days_of_m = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
slices_of_year = np.dot(48,day_of_year)
for i in np.arange(1,len(slices_of_year)):
    slices_of_year[i] = slices_of_year[i] + slices_of_year[i-1]

# Determine the data length
pid = np.ones(totlen)
idx = 0
for i in np.arange(0,totlen):
    idx = idx + 1
    pid[i] = idx

yr = np.ones(totlen)
ptyear = 0
for i in np.arange(0,totlen):
    yr[i] = totyrrange[ptyear]
    if (i >= slices_of_year[ptyear] - 1):
        ptyear = ptyear + 1
        
doy = np.ones(totlen)
hr = np.ones(totlen)
accuday = 0
for i in np.arange(0,len(day_of_year)):
    ptday = 0
    for j in np.arange(0,day_of_year[i]):
        ptday = ptday + 1
        for k in np.arange(0,48):
            doy[k+accuday*48] = ptday
            hr[k+accuday*48] = k/2. + 0.25
        accuday = accuday + 1


#======================================================
## Read the observation of FCH4
#======================================================
varname1 = 'FCH4'
varname2 = 'FCH4_CMB'

# Aeriflux
# Data are from different sources 
# so we extract observation from each site separately

# US-PFa site data (AMF BASE product) 
# hrly dataset
# Unit = nmol m-2 s-1
filename = 'AMF_US-PFa_BASE_HR_10-4.csv'
data = pd.read_csv(filename, encoding='iso-8859-1', skiprows=[0, 1]) #, index_col='ProfileID', skiprows=[1])
yrrange = 1995 + np.arange(23)   # 23 years data
res = 1   # Unit: hour
CH4 = data.FCH4_1_1_1.as_matrix()
obs_date = (data.TIMESTAMP_START.as_matrix() + data.TIMESTAMP_END.as_matrix())/2.
totlen = 403248
# Coordinate the time period of FCH4 timeseries for US-PFa
ch4_uspfa = np.ones(totlen)
for i in np.arange(0,len(CH4)):
    for j in np.arange(0,2):
        ch4_uspfa[2*i+j] = CH4[i]

# US-WPT site data (OLD AMF data accessed from ORNL FTP site)
# 30m dataset
# Unit = umol m-2 s-1
sitename = ['USWPT']
ch4_uswpt = np.ones(totlen)*(-9999.)
for i in sitename:
    cmd = "ls -1 AMF_"+i+"_*.csv"
    flist = subprocess.check_output(cmd, shell=True).splitlines()
    yrrange=np.zeros(len(flist))
    for j in np.arange(0,len(flist)):
        yrrange[j] = int(flist[j][10:14])
        data = pd.read_csv(flist[j], encoding='iso-8859-1')
        CH4 = 0.001 * data.FCH4.as_matrix()   # convert to nmol
        CH4[CH4 < -5.] = -9999.0
        obs_date = data.TIMESTAMP.as_matrix() + 1500      
        # Coordinate the time period of FCH4 timeseries for US-WPT
        yr_site = np.floor(obs_date/1e10)
        dd = np.floor((obs_date/1e10-np.floor(obs_date/1e10))*1e4)
        hh = np.round((obs_date/1e6 -np.floor(obs_date/1e6))*1e2)
        mm = np.round((obs_date/1e4 -np.floor(obs_date/1e4))*1e2)/60.
        mo = np.floor(dd/1e2).astype("int")
        hr_site=hh+mm
        # Get the first time step of the timeseries from file
        sitelen = len(CH4)
        pt = 0
        yr_site_pt = yr_site[pt].astype("int")
        doy_site_pt = int((mo[pt]-1)*days_of_m[mo[pt]] + ((dd[pt]/1e2-np.floor(dd[pt]/1e2))*1e2))
        hr_site_pt = hr_site[pt]
        # Place into the array
        for k in np.arange(0,totlen):
            if(yr_site_pt == yr[k] and doy_site_pt == doy[k] and (hr_site_pt-hr[k])<1e-6):
                idx = k
                ch4_uswpt[idx:idx+sitelen] = CH4


# US-ORv site data (OLD AMF data accessed from ORNL FTP site)
# 30m dataset
# Unit = umol m-2 s-1
sitename = ['ORW']
ch4_usorv = np.ones(totlen)*(-9999.)
for i in sitename:
    cmd = "ls -1 "+i+"_*.csv"
    flist = subprocess.check_output(cmd, shell=True).splitlines()
    yrrange=np.zeros(len(flist))
    for j in np.arange(0,len(flist)):
        yrrange[j] = int(flist[j][4:8])
        data = pd.read_csv(flist[j], encoding='iso-8859-1', skiprows=[0, 1, 2, 4])
        CH4 = 1000. * data.FCH4.as_matrix()      # convert to nmol
        obs_date = 10000000 * data.YEAR.as_matrix() + 10000 * data.DOY.as_matrix() \
             + 100 * np.floor(data.HRMIN.as_matrix()) + 60 * (data.HRMIN.as_matrix() \
             - np.floor(data.HRMIN.as_matrix())) + 15
        # Coordinate the time period of FCH4 timeseries for US-WPT
        yr_site = np.floor(obs_date/1e7)
        dd = np.floor((obs_date/1e7-np.floor(obs_date/1e7))*1e3)
        hh = np.round((obs_date/1e4 -np.floor(obs_date/1e4))*1e2)
        mm = np.round((obs_date/1e2 -np.floor(obs_date/1e2))*1e2)/60.
        mo = np.floor(dd/1e2).astype("int")
        hr_site=hh+mm
        # Get the first time step of the timeseries from file
        sitelen = len(CH4)
        pt = 0
        yr_site_pt = yr_site[pt].astype("int")
        doy_site_pt = int(dd[pt])
        hr_site_pt = hr_site[pt]
        # Place into the array
        for k in np.arange(0,totlen):
            if(yr_site_pt == yr[k] and doy_site_pt == doy[k] and (hr_site_pt-hr[k])<1e-6):
                idx = k
                ch4_usorv[idx:idx+sitelen] = CH4
            
# US-Bes site data (OLD AMF data accessed from ORNL FTP site)
# 30m dataset
# Unit = umol m-2 s-1
sitename = ['US-Bes']
ch4_usbes = np.ones(totlen)*(-9999.)
for i in sitename:
    cmd = "ls -1 "+i+"_*.csv"
    flist = subprocess.check_output(cmd, shell=True).splitlines()
    yrrange=np.zeros(len(flist))
    for j in np.arange(0,len(flist)):
        yrrange[j] = int(flist[j][7:11])
        data = pd.read_csv(flist[j], encoding='iso-8859-1', skiprows=[1])
        CH4 = 1000. * data.FCH4.as_matrix()   # convert to nmol
        obs_date = data.TIMESTAMP.as_matrix()
        # Coordinate the time period of FCH4 timeseries for US-WPT
        yr_site = np.floor(obs_date/1e8)
        dd = np.floor((obs_date/1e8 -np.floor(obs_date/1e8))*1e4)
        hh = np.round((obs_date/1e4 -np.floor(obs_date/1e4))*1e2)
        mm = np.round((obs_date/1e2 -np.floor(obs_date/1e2))*1e2)/60.
        mo = np.floor(dd/1e2).astype("int")
        hr_site=hh+mm
        # Get the first time step of the timeseries from file
        sitelen = len(CH4)
        pt = 0
        yr_site_pt = yr_site[pt].astype("int")
        doy_site_pt = int((mo[pt]-1)*days_of_m[mo[pt]] + ((dd[pt]/1e2-np.floor(dd[pt]/1e2))*1e2))
        hr_site_pt = hr_site[pt]
        # Place into the array
        for k in np.arange(0,totlen):
            if(yr_site_pt == yr[k] and doy_site_pt == doy[k] and (hr_site_pt-hr[k])<1e-6):
                idx = k
                ch4_usbes[idx:idx+sitelen] = CH4
        
        
# US-IVO site data (OLD AMF data accessed from ORNL FTP site)
# 30m dataset
# Unit = umol m-2 s-1
sitename = ['US-IVO']
ch4_usivo = np.ones(totlen)*(-9999.)
for i in sitename:
    cmd = "ls -1 "+i+"_*.csv"
    flist = subprocess.check_output(cmd, shell=True).splitlines()
    yrrange=np.zeros(len(flist))
    for j in np.arange(0,len(flist)):
        yrrange[j] = int(flist[j][7:11])
        data = pd.read_csv(flist[j], encoding='iso-8859-1', skiprows=[1])
        CH4 = 1000. * data.FCH4.as_matrix()   # convert to nmol
        obs_date = data.TIMESTAMP.as_matrix()
        # Coordinate the time period of FCH4 timeseries for US-WPT
        yr_site = np.floor(obs_date/1e8)
        dd = np.floor((obs_date/1e8 -np.floor(obs_date/1e8))*1e4)
        hh = np.round((obs_date/1e4 -np.floor(obs_date/1e4))*1e2)
        mm = np.round((obs_date/1e2 -np.floor(obs_date/1e2))*1e2)/60.
        mo = np.floor(dd/1e2).astype("int")
        hr_site=hh+mm
        # Get the first time step of the timeseries from file
        sitelen = len(CH4)
        pt = 0
        yr_site_pt = yr_site[pt].astype("int")
        doy_site_pt = int((mo[pt]-1)*days_of_m[mo[pt]] + ((dd[pt]/1e2-np.floor(dd[pt]/1e2))*1e2))
        hr_site_pt = hr_site[pt]
        # Place into the array
        for k in np.arange(0,totlen):
            if(yr_site_pt == yr[k] and doy_site_pt == doy[k] and (hr_site_pt-hr[k])<1e-6):
                idx = k
                ch4_usivo[idx:idx+sitelen] = CH4
            
# EFDC EC data
# 30m dataset
# Totally 4 sites are available
# Unit = umol m-2 s-1
sitename = ['DESfN', 'FIHyy', 'NLHor', 'RUCh2']
for i in sitename:
    if(i == 'DESfN'):
        ch4_desfn = np.ones(totlen)*(-9999.)
    if(i == 'FIHyy'):
        ch4_fihyy = np.ones(totlen)*(-9999.)
    if(i == 'NLHor'):
        ch4_nlhor = np.ones(totlen)*(-9999.)
    if(i == 'RUCh2'):
        ch4_ruche = np.ones(totlen)*(-9999.)
    cmd = "ls -1 EFDC_*"+i+"_*30m.txt"
    flist = subprocess.check_output(cmd, shell=True).splitlines()
    yrrange=np.zeros(len(flist))
    for j in np.arange(0,len(flist)):
        yrrange[j] = int(flist[j][18:22])
        data = pd.read_csv(flist[j], encoding='iso-8859-1')
        CH4 = data.FCH4.as_matrix()
        timest = data.TIMESTAMP_START.as_matrix()
        timeend = data.TIMESTAMP_END.as_matrix()
        sel = ((timeend/1e2-np.floor(timeend/1e2))*1e2 == 0)
        timeend[sel] = timeend[sel] - 40
        sel = (np.round((timeend/1e4-np.floor(timeend/1e4))*1e4) == 9960)
        timeend[sel] = timeend[sel] - 7600
        obs_date = np.floor(timest/1e2)*1e2 + (timest/1e2-np.floor(timest/1e2) + \
                                               timeend/1e2-np.floor(timeend/1e2))*1e2/2.

        # Coordinate the time period of FCH4 timeseries for US-WPT
        yr_site = np.floor(obs_date/1e8)
        dd = np.floor((obs_date/1e8 -np.floor(obs_date/1e8))*1e4)
        hh = np.round((obs_date/1e4 -np.floor(obs_date/1e4))*1e2)
        mm = np.round((obs_date/1e2 -np.floor(obs_date/1e2))*1e2)/60.
        mo = np.floor(dd/1e2).astype("int")
        hr_site=hh+mm
        # Get the first time step of the timeseries from file
        sitelen = len(CH4)
        pt = 0
        yr_site_pt = yr_site[pt].astype("int")
        doy_site_pt = int((mo[pt]-1)*days_of_m[mo[pt]] + ((dd[pt]/1e2-np.floor(dd[pt]/1e2))*1e2))
        hr_site_pt = hr_site[pt]
        # Place into the array
        for k in np.arange(0,totlen):
            if(yr_site_pt == yr[k] and doy_site_pt == doy[k] and (hr_site_pt-hr[k])<1e-6):
                idx = k
                if(i == 'DESfN'):
                    ch4_desfn[idx:idx+sitelen] = CH4
                if(i == 'FIHyy'):
                    ch4_fihyy[idx:idx+sitelen] = CH4
                if(i == 'NLHor'):
                    ch4_nlhor[idx:idx+sitelen] = CH4
                if(i == 'RUCh2'):
                    ch4_ruche[idx:idx+sitelen] = CH4
                    
# AU-RFi site data (Ridge Field, from Ozflux)
# 30min dataset
# Unit = umol m-2 s-1
sitename = ['AU-RFi']
varname = 'Fch4'
ch4_aurfi = np.ones(totlen)*(-9999.)
for i in sitename:
    cmd = "ls -1 Ridgefield_*.nc"
    flist = subprocess.check_output(cmd, shell=True).splitlines()
    yrrange=np.zeros(len(flist))
    for j in np.arange(0,len(flist)):
        yrrange[j] = int(flist[j][11:15])
        nc = Dataset(flist[j], 'r', format ='NETCDF4_CLASSIC')
        yr_site = np.squeeze(nc.variables['Year'][:])  # extract/copy the data
        dd = np.squeeze(nc.variables['Ddd'][:])
        hh = np.squeeze(nc.variables['Hour'][:])
        mm = np.squeeze(nc.variables['Minute'][:])
        CH4 = 1000. * np.squeeze(nc.variables['Fch4'][:]) # shape is lat, lon as shown above
        CH4[CH4 > 500.] = -9999.0
        nc.close()
        # Coordinate the time period of FCH4 timeseries for US-WPT
        mm = mm/60.
        hr_site=hh+mm
        # Get the first time step of the timeseries from file
        sitelen = len(CH4)
        pt = 0
        yr_site_pt = yr_site[pt].astype("int")
        doy_site_pt = dd[pt].astype("int")
        hr_site_pt = hr_site[pt]
        # Place into the array
        for k in np.arange(0,totlen):
            if(yr_site_pt == yr[k] and doy_site_pt == doy[k] and (hr_site_pt-hr[k])<1e-6):
                idx = k
                ch4_aurfi[idx:idx+sitelen] = CH4
                    
# PH-IRI site data (Ridge Field, from Ozflux)
# 30min dataset
# Unit = umol m-2 s-1
sitename = ['PH-IRI']
varname = 'CH4'
ch4_phiri = np.ones(totlen)*(-9999.)
for i in sitename:
    cmd = "ls -1 FxMt_IRI-FL_*.csv"
    flist = subprocess.check_output(cmd, shell=True).splitlines()
    yrrange=np.zeros(len(flist))
    for j in np.arange(0,len(flist)):
        yrrange[j] = int(flist[j][12:16])
        data = pd.read_csv(flist[j], encoding='iso-8859-1', skiprows=[1])
        CH4 = 1000. * data.CH4.as_matrix()   # convert to nmol
        # Coordinate the time period of FCH4 timeseries for US-WPT
        yr_site = data.Year.as_matrix()
        dd = data.DOY.as_matrix()
        hh = np.round(data.TIME.as_matrix()/1e2)
        mm = np.round((data.TIME.as_matrix()/1e2 -np.floor(data.TIME.as_matrix()/1e2))*1e2)/60. - 0.25
        hr_site=hh+mm
        # Get the first time step of the timeseries from file
        sitelen = len(CH4)
        pt = 0
        yr_site_pt = yr_site[pt].astype("int")
        doy_site_pt = dd[pt]
        hr_site_pt = hr_site[pt]
        # Place into the array
        for k in np.arange(0,totlen):
            if(yr_site_pt == yr[k] and doy_site_pt == doy[k] and (hr_site_pt-hr[k])<1e-6):
                idx = k
                ch4_phiri[idx:idx+sitelen] = CH4

# Read in four old sites.
# US-Twt
# transfer from nmol m-2 s-1 to umol m-2 s-1
sitename = ['US-Twt']
varname = "FCH4"
ch4_ustwt = np.ones(totlen)*(-9999.)
fname = "US-Twt_WT.nc"
nc = Dataset(fname, 'r', format ='NETCDF4_CLASSIC')
yr_site = 2010  # extract/copy the data
dd = 1
hh = 0
mm = 0.25
CH4 = 0.001 * np.squeeze(nc.variables['FCH4'][:]) # shape is lat, lon as shown above
# Do not filter the measurment from US-Twt site.
#CH4[CH4 < -500.] = -9999.0   
nc.close()
hr_site=hh+mm
# Get the first time step of the timeseries from file
sitelen = len(CH4)
yr_site_pt = yr_site
doy_site_pt = dd
hr_site_pt = hr_site
# Place into the array
for k in np.arange(0,totlen):
    if(yr_site_pt == yr[k] and doy_site_pt == doy[k] and (hr_site_pt-hr[k])<1e-6):
        idx = k
        ch4_ustwt[idx:idx+sitelen] = CH4

# US-Myb
# transfer from nmol m-2 s-1 to umol m-2 s-1
sitename = ['US-Myb']
varname = "FCH4"
ch4_usmyb = np.ones(totlen)*(-9999.)
fname = "US-Myb_WT.nc"
nc = Dataset(fname, 'r', format ='NETCDF4_CLASSIC')
yr_site = 2011  # extract/copy the data
dd = 1
hh = 0
mm = 0.25
CH4 = 0.001 * np.squeeze(nc.variables['FCH4'][:]) # shape is lat, lon as shown above
CH4[CH4 < -5.] = -9999.0
nc.close()
hr_site=hh+mm
# Get the first time step of the timeseries from file
sitelen = len(CH4)
yr_site_pt = yr_site
doy_site_pt = dd
hr_site_pt = hr_site
# Place into the array
for k in np.arange(0,totlen):
    if(yr_site_pt == yr[k] and doy_site_pt == doy[k] and (hr_site_pt-hr[k])<1e-6):
        idx = k
        ch4_usmyb[idx:idx+sitelen] = CH4

# US-CRT
# transfer from nmol m-2 s-1 to umol m-2 s-1
sitename = ['US-CRT']
varname = "FCH4"
ch4_uscrt = np.ones(totlen)*(-9999.)
fname = "US-CRT_WT.nc"
nc = Dataset(fname, 'r', format ='NETCDF4_CLASSIC')
yr_site = 2011  # extract/copy the data
dd = 1
hh = 0
mm = 0.25
CH4 = 0.001 * np.squeeze(nc.variables['FCH4'][:]) # shape is lat, lon as shown above
CH4[CH4 < -5.] = -9999.0
nc.close()
hr_site=hh+mm
# Get the first time step of the timeseries from file
sitelen = len(CH4)
yr_site_pt = yr_site
doy_site_pt = dd
hr_site_pt = hr_site
# Place into the array
for k in np.arange(0,totlen):
    if(yr_site_pt == yr[k] and doy_site_pt == doy[k] and (hr_site_pt-hr[k])<1e-6):
        idx = k
        ch4_uscrt[idx:idx+sitelen] = CH4
        
# US-Tw1
# transfer from nmol m-2 s-1 to umol m-2 s-1
sitename = ['US-Tw1']
varname = "FCH4"
ch4_ustw1 = np.ones(totlen)*(-9999.)
fname = "US-Tw1_WT.nc"
nc = Dataset(fname, 'r', format ='NETCDF4_CLASSIC')
yr_site = 2013  # extract/copy the data
dd = 1
hh = 0
mm = 0.25
CH4 = 0.001 * np.squeeze(nc.variables['FCH4'][:]) # shape is lat, lon as shown above
CH4[CH4 < -5.] = -9999.0
nc.close()
hr_site=hh+mm
# Get the first time step of the timeseries from file
sitelen = len(CH4)
yr_site_pt = yr_site
doy_site_pt = dd
hr_site_pt = hr_site
# Place into the array
for k in np.arange(0,totlen):
    if(yr_site_pt == yr[k] and doy_site_pt == doy[k] and (hr_site_pt-hr[k])<1e-6):
        idx = k
        ch4_ustw1[idx:idx+sitelen] = CH4
        
# Create dataframe to combine all observations
d = {'ID': pid, 'YEAR': yr, 'DOY': doy, 'TIME': hr, 'USPFa': ch4_uspfa, 'USWPT': ch4_uswpt, \
     'USORv': ch4_usorv, 'USBes': ch4_usbes, 'USIVO': ch4_usivo, 'DESfN': ch4_desfn, \
     'FIHyy': ch4_fihyy, 'NLHor': ch4_nlhor, 'RUCh2': ch4_ruche, 'AURFi': ch4_aurfi, \
     'PHIRI': ch4_phiri, 'USTwt': ch4_ustwt, 'USMyb': ch4_usmyb, 'USCRT': ch4_uscrt, \
     'USTw1': ch4_ustw1}

methane = pd.DataFrame(data=d)
# Write the data as CSV file
methane.to_csv('site_methane.csv')

# ===============================================
## Checkout chamber data later
# ===============================================
# EFDC noon-time CH4 Chamber measurement
# daily dataset
# Totally 6 sites are available
# Unit?
if(0):   # Right now commented
    sitename = ['FI-Hyy', 'FI-Kaa', 'IT-CA1', 'IT-CA2', 'IT-CA3', 'IT-Ro2']
    for i in sitename:
        cmd = "ls -1 "+i+"_Fluxes*.txt"
        flist = subprocess.check_output(cmd, shell=True).splitlines()
        yrrange=np.zeros(len(flist))
        for j in np.arange(0,len(flist)):
            yrrange[j] = int(flist[j][14:18])
            data = pd.read_csv(flist[j], encoding='iso-8859-1')
            #CH4 = data.FCH4.as_matrix()
            #obs_date = (data.TIMESTAMP_START.as_matrix() + data.TIMESTAMP_END.as_matrix())/2.

            

In [3]:
#======================================================
## Organize the data
#======================================================
totlen = 403248     # From 1995 to 2017
totyrrange = 1995 + np.arange(23)
day_of_year = [365, 366, 365, 365, 365, 366, 365, 365, 365, 366, 365, \
               365, 365, 366, 365, 365, 365, 366, 365, 365, 365, 366, 365]   # DOY from 1995 to 2017
days_of_m = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
slices_of_year = np.dot(48,day_of_year)
for i in np.arange(1,len(slices_of_year)):
    slices_of_year[i] = slices_of_year[i] + slices_of_year[i-1]

# Determine the data length
pid = np.ones(totlen)
idx = 0
for i in np.arange(0,totlen):
    idx = idx + 1
    pid[i] = idx

yr = np.ones(totlen)
ptyear = 0
for i in np.arange(0,totlen):
    yr[i] = totyrrange[ptyear]
    if (i >= slices_of_year[ptyear] - 1):
        ptyear = ptyear + 1
        
doy = np.ones(totlen)
hr = np.ones(totlen)
accuday = 0
for i in np.arange(0,len(day_of_year)):
    ptday = 0
    for j in np.arange(0,day_of_year[i]):
        ptday = ptday + 1
        for k in np.arange(0,48):
            doy[k+accuday*48] = ptday
            hr[k+accuday*48] = k/2. + 0.25
        accuday = accuday + 1



In [4]:
methane.to_csv('site_methane.csv')

In [ ]:
np.savetxt( "sss", ch4_ustwt)

In [ ]:
#======================================================
## Organize the data
#======================================================
totlen = 403248     # From 1995 to 2017
totyrrange = 1995 + np.arange(23)
day_of_year = [365, 366, 365, 365, 365, 366, 365, 365, 365, 366, 365, \
               365, 365, 366, 365, 365, 365, 366, 365, 365, 365, 366, 365]   # DOY from 1995 to 2017
days_of_m = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
slices_of_year = np.dot(48,day_of_year)
for i in np.arange(1,len(slices_of_year)):
    slices_of_year[i] = slices_of_year[i] + slices_of_year[i-1]


# Site info: lat, lon and data availability
uspfa_loc = [45.9459, -90.2723]
uspfa_yr  = [2010, 2014]
uswpt_loc = [41.464639, -82.996157]
uswpt_yr  = [2011, 2013]
usorv_loc = [40.0201, -83.0183]
usorv_yr  = [2011, 2012]
usbes_loc = [71.2809, -156.5965]
usbes_yr  = [2013, 2014]
usivo_loc = [68.4865, -155.7503]
usivo_yr  = [2013, 2014]
desfn_loc = [47.8064, 11.3275]
desfn_yr  = [2012, 2015]
fihyy_loc = [61.8474, 24.2948]
fihyy_yr  = [2012, 2013]
nlhor_loc = [52.24035, 5.071301]
nlhor_yr  = [2006, 2009]
ruche_loc = [68.61304, 161.34143]
ruche_yr  = [2014, 2015]
aurfi_loc = [-32.5061, 116.9668]
aurfi_yr  = [2016, 2017]
phiri_loc = [14.1412, 121.2653]
phiri_yr  = [2013, 2014]


methane = pd.read_csv('site_methane.csv')

# Read in the model output and extract the corresponding grid point to compare with the observation.
#fncname = 'Global_1DSBGC.bgc-yearly-2d_1920.nc'
#nc = Dataset(fncname, 'r', format ='NETCDF4_CLASSIC')
#lats = nc.variables['lat'][:]  # extract/copy the data
#lons = nc.variables['lon'][:]
#ch4_isam = nc.variables['ch4_flux'][:] # shape is lat, lon as shown above
#nc.close()

# Read site level simulation resylts
reader = pd.read_csv('uspfa_ch4.txt')

uspfa_isam_yr = [uspfa_yr[0] - 1990, uspfa_yr[1] - 1990]
uspfa_isam = reader.FCH4.as_matrix()
uspfa_isam = uspfa_isam[uspfa_isam_yr[0]*365:(uspfa_isam_yr[1])*365]
uspfa_isam = uspfa_isam*1e9/(12*24*3600)
# Calculate daily mean
uspfa_obs_yr = [uspfa_yr[0] - 1995, uspfa_yr[1] - 1995]
uspfa_obs = methane.USPFa[slices_of_year[uspfa_obs_yr[0]]:slices_of_year[uspfa_obs_yr[1]]]
uspfa_obs[uspfa_obs<-100] = np.float("nan")
uspfa_obs_daily = np.ones(len(uspfa_isam))*float("nan")
for j in np.arange(0,(uspfa_obs_yr[1]-uspfa_obs_yr[0])):
    for i in np.arange(0,365):
        uspfa_obs_daily[365*j+i] = np.nanmean(uspfa_obs[j*17520+i*48:j*17520+(i+1)*48])

# ========================================================
# Plot the modeled CH4 fluxes against EC observation
# ========================================================
Xobs = np.arange(0,len(uspfa_obs_daily))
Yobs = uspfa_obs_daily
Xmod = Xobs
Ymod = uspfa_isam
tit = "CH4 flux"
path = "./uspfa.jpg"

status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path)


In [ ]:
# Read site level simulation resylts
reader = pd.read_csv('usbes_ch4.txt')

usbes_isam_yr = [usbes_yr[0] - 1990, usbes_yr[1] - 1990]
usbes_isam = reader.FCH4.as_matrix()
usbes_isam = usbes_isam[usbes_isam_yr[0]*365:(usbes_isam_yr[1])*365]
usbes_isam = usbes_isam*1e9/(12*24*3600)
# Calculate daily mean
usbes_obs_yr = [usbes_yr[0] - 1995, usbes_yr[1] - 1995]
usbes_obs = methane.USBes[slices_of_year[usbes_obs_yr[0]]:slices_of_year[usbes_obs_yr[1]]]
usbes_obs[usbes_obs<-100] = np.float("nan")
usbes_obs_daily = np.ones(len(usbes_isam))*float("nan")
for j in np.arange(0,(usbes_obs_yr[1]-usbes_obs_yr[0])):
    for i in np.arange(0,365):
        usbes_obs_daily[365*j+i] = np.nanmean(usbes_obs[j*17520+i*48:j*17520+(i+1)*48])

# ========================================================
# Plot the modeled CH4 fluxes against EC observation
# ========================================================
Xobs = np.arange(0,len(usbes_obs_daily))
Yobs = usbes_obs_daily/10.
Xmod = Xobs
Ymod = usbes_isam
tit = "CH4 flux"
path = "./usbes.jpg"

status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path)

In [ ]:
# Read site level simulation resylts
reader = pd.read_csv('usivo_ch4.txt')

usivo_isam_yr = [usivo_yr[0] - 1990, usivo_yr[1] - 1990]
usivo_isam = reader.FCH4.as_matrix()
usivo_isam = usivo_isam[usivo_isam_yr[0]*365:(usivo_isam_yr[1])*365]
usivo_isam = usivo_isam*1e9/(12*24*3600)
# Calculate daily mean
usivo_obs_yr = [usivo_yr[0] - 1995, usivo_yr[1] - 1995]
usivo_obs = methane.USIVO[slices_of_year[usivo_obs_yr[0]]:slices_of_year[usivo_obs_yr[1]]]
usivo_obs[usivo_obs<-100] = np.float("nan")
usivo_obs_daily = np.ones(len(usivo_isam))*float("nan")
for j in np.arange(0,(usivo_obs_yr[1]-usivo_obs_yr[0])):
    for i in np.arange(0,365):
        usivo_obs_daily[365*j+i] = np.nanmean(usivo_obs[j*17520+i*48:j*17520+(i+1)*48])

# ========================================================
# Plot the modeled CH4 fluxes against EC observation
# ========================================================
Xobs = np.arange(0,len(usivo_obs_daily))
Yobs = usivo_obs_daily/10.
Xmod = Xobs
Ymod = usivo_isam
tit = "CH4 flux"
path = "./usivo.jpg"

status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path)

In [ ]:
# Read site level simulation resylts
reader = pd.read_csv('uswpt_ch4.txt')

uswpt_isam_yr = [uswpt_yr[0] - 1990, uswpt_yr[1] - 1990]
uswpt_isam = reader.FCH4.as_matrix()
uswpt_isam = uswpt_isam[uswpt_isam_yr[0]*365:(uswpt_isam_yr[1])*365]
uswpt_isam = uswpt_isam*1e9/(12*24*3600)
# Calculate daily mean
uswpt_obs_yr = [uswpt_yr[0] - 1995, uswpt_yr[1] - 1995]
uswpt_obs = methane.USWPT[slices_of_year[uswpt_obs_yr[0]]:slices_of_year[uswpt_obs_yr[1]]]
uswpt_obs[uswpt_obs<-100] = np.float("nan")
uswpt_obs_daily = np.ones(len(uswpt_isam))*float("nan")
for j in np.arange(0,(uswpt_obs_yr[1]-uswpt_obs_yr[0])):
    for i in np.arange(0,365):
        uswpt_obs_daily[365*j+i] = np.nanmean(uswpt_obs[j*17520+i*48:j*17520+(i+1)*48])

# ========================================================
# Plot the modeled CH4 fluxes against EC observation
# ========================================================
Xobs = np.arange(0,len(uswpt_obs_daily))
Yobs = uswpt_obs_daily*2/10000.
Xmod = Xobs
Ymod = uswpt_isam
tit = "CH4 flux"
path = "./uswpt.jpg"

status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path)

In [ ]:
# Read site level simulation resylts
reader = pd.read_csv('usorv_ch4.txt')

usorv_isam_yr = [usorv_yr[0] - 1990, usorv_yr[1] - 1990]
usorv_isam = reader.FCH4.as_matrix()
usorv_isam = usorv_isam[usorv_isam_yr[0]*365:(usorv_isam_yr[1])*365]
usorv_isam = usorv_isam*1e9/(12*24*3600)
# Calculate daily mean
usorv_obs_yr = [usorv_yr[0] - 1995, usorv_yr[1] - 1995]
usorv_obs = methane.USORv[slices_of_year[usorv_obs_yr[0]]:slices_of_year[usorv_obs_yr[1]]]
usorv_obs[usorv_obs<-100] = np.float("nan")
usorv_obs_daily = np.ones(len(usorv_isam))*float("nan")
for j in np.arange(0,(usorv_obs_yr[1]-usorv_obs_yr[0])):
    for i in np.arange(0,365):
        usorv_obs_daily[365*j+i] = np.nanmean(usorv_obs[j*17520+i*48:j*17520+(i+1)*48])

# ========================================================
# Plot the modeled CH4 fluxes against EC observation
# ========================================================
Xobs = np.arange(0,len(usorv_obs_daily))
Yobs = usorv_obs_daily/10.
Xmod = Xobs
Ymod = usorv_isam
tit = "CH4 flux"
path = "./usorv.jpg"

status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path)

In [ ]:
# Read site level simulation resylts
reader = pd.read_csv('desfn_ch4.txt')

desfn_isam_yr = [desfn_yr[0] - 1990, desfn_yr[1] - 1990]
desfn_isam = reader.FCH4.as_matrix()
desfn_isam = desfn_isam[desfn_isam_yr[0]*365:(desfn_isam_yr[1])*365]
desfn_isam = desfn_isam*1e9/(12*24*3600)
# Calculate daily mean
desfn_obs_yr = [desfn_yr[0] - 1995, desfn_yr[1] - 1995]
desfn_obs = methane.DESfN[slices_of_year[desfn_obs_yr[0]]:slices_of_year[desfn_obs_yr[1]]]
desfn_obs[desfn_obs<-100] = np.float("nan")
desfn_obs_daily = np.ones(len(desfn_isam))*float("nan")
for j in np.arange(0,(desfn_obs_yr[1]-desfn_obs_yr[0])):
    for i in np.arange(0,365):
        desfn_obs_daily[365*j+i] = np.nanmean(desfn_obs[j*17520+i*48:j*17520+(i+1)*48])

# ========================================================
# Plot the modeled CH4 fluxes against EC observation
# ========================================================
Xobs = np.arange(0,len(desfn_obs_daily))
Yobs = desfn_obs_daily*100.
Xmod = Xobs
Ymod = desfn_isam
tit = "CH4 flux"
path = "./desfn.jpg"

status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path)

In [ ]:
# Read site level simulation resylts
reader = pd.read_csv('fihyy_ch4.txt')

fihyy_isam_yr = [fihyy_yr[0] - 1990, fihyy_yr[1] - 1990]
fihyy_isam = reader.FCH4.as_matrix()
fihyy_isam = fihyy_isam[fihyy_isam_yr[0]*365:(fihyy_isam_yr[1])*365]
fihyy_isam = fihyy_isam*1e9/(12*24*3600)
# Calculate daily mean
fihyy_obs_yr = [fihyy_yr[0] - 1995, fihyy_yr[1] - 1995]
fihyy_obs = methane.FIHyy[slices_of_year[fihyy_obs_yr[0]]:slices_of_year[fihyy_obs_yr[1]]]
fihyy_obs[fihyy_obs<-100] = np.float("nan")
fihyy_obs_daily = np.ones(len(fihyy_isam))*float("nan")
for j in np.arange(0,(fihyy_obs_yr[1]-fihyy_obs_yr[0])):
    for i in np.arange(0,365):
        fihyy_obs_daily[365*j+i] = np.nanmean(fihyy_obs[j*17520+i*48:j*17520+(i+1)*48])

# ========================================================
# Plot the modeled CH4 fluxes against EC observation
# ========================================================
Xobs = np.arange(0,len(fihyy_obs_daily))
Yobs = fihyy_obs_daily
Xmod = Xobs
Ymod = fihyy_isam
tit = "CH4 flux"
path = "./fihyy.jpg"

status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path)

In [ ]:
# Read site level simulation resylts
reader = pd.read_csv('nlhor_ch4.txt')

nlhor_isam_yr = [nlhor_yr[0] - 1990, nlhor_yr[1] - 1990]
nlhor_isam = reader.FCH4.as_matrix()
nlhor_isam = nlhor_isam[nlhor_isam_yr[0]*365:(nlhor_isam_yr[1])*365]
nlhor_isam = nlhor_isam*1e9/(12*24*3600)
# Calculate daily mean
nlhor_obs_yr = [nlhor_yr[0] - 1995, nlhor_yr[1] - 1995]
nlhor_obs = methane.NLHor[slices_of_year[nlhor_obs_yr[0]]:slices_of_year[nlhor_obs_yr[1]]]
nlhor_obs[nlhor_obs<-100] = np.float("nan")
nlhor_obs_daily = np.ones(len(nlhor_isam))*float("nan")
for j in np.arange(0,(nlhor_obs_yr[1]-nlhor_obs_yr[0])):
    for i in np.arange(0,365):
        nlhor_obs_daily[365*j+i] = np.nanmean(nlhor_obs[j*17520+i*48:j*17520+(i+1)*48])

# ========================================================
# Plot the modeled CH4 fluxes against EC observation
# ========================================================
Xobs = np.arange(0,len(nlhor_obs_daily))
Yobs = nlhor_obs_daily*1000.
Xmod = Xobs
Ymod = nlhor_isam
tit = "CH4 flux"
path = "./nlhor.jpg"

status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path)

In [ ]:
# Read site level simulation resylts
reader = pd.read_csv('ruch2_ch4.txt')

ruche_isam_yr = [ruche_yr[0] - 1990, ruche_yr[1] - 1990]
ruche_isam = reader.FCH4.as_matrix()
ruche_isam = ruche_isam[ruche_isam_yr[0]*365:(ruche_isam_yr[1])*365]
ruche_isam = ruche_isam*1e9/(12*24*3600)
# Calculate daily mean
ruche_obs_yr = [ruche_yr[0] - 1995, ruche_yr[1] - 1995]
ruche_obs = methane.RUChe[slices_of_year[ruche_obs_yr[0]]:slices_of_year[ruche_obs_yr[1]]]
ruche_obs[ruche_obs<-100] = np.float("nan")
ruche_obs_daily = np.ones(len(ruche_isam))*float("nan")
for j in np.arange(0,(ruche_obs_yr[1]-ruche_obs_yr[0])):
    for i in np.arange(0,365):
        ruche_obs_daily[365*j+i] = np.nanmean(ruche_obs[j*17520+i*48:j*17520+(i+1)*48])

# ========================================================
# Plot the modeled CH4 fluxes against EC observation
# ========================================================
Xobs = np.arange(0,len(ruche_obs_daily))
Yobs = ruche_obs_daily
Xmod = Xobs
Ymod = ruche_isam
tit = "CH4 flux"
path = "./ruche.jpg"

status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path)

In [ ]:
ruche_yr  = [2014, 2015]

In [ ]:
# ========================================================
# Plot the modeled CH4 fluxes against EC observation
# ========================================================
Xobs = np.arange(0,len(uspfa_obs_daily))
Yobs = uspfa_obs_daily
Xmod = Xobs
Ymod = uspfa_isam
tit = "CH4 flux"
path = "./uspfa.jpg"

status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path)


In [ ]:
usorv_yr  = [2011, 2012]

In [ ]:
# ========================================================
# Plot the modeled CH4 fluxes against EC observation
# ========================================================
Xobs = np.arange(0,totlen)
Yobs = methane.USPFa
Yobs[Yobs < -1000.] = float('nan')
Xmod = np.arange(0,totlen)
Ymod = uspfa_isam * np.ones(totlen)
tit = "CH4 flux"
path = "./uspfa.jpg"

status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path)

In [ ]:
# ========================================================
# Plot the site location on the map
# ========================================================
# Site info: lat and lon
usmyb_loc = [38.0498, -121.7651]
ustwt_loc = [38.10867, -121.6530]
uscrt_loc = [41.6285, -83.3471]
uspfa_loc = [45.9459, -90.2723]
uswpt_loc = [41.464639, -82.996157]
usorv_loc = [40.0201, -83.0183]
usbes_loc = [71.2809, -156.5965]
usivo_loc = [68.4865, -155.7503]
desfn_loc = [47.8064, 11.3275]
fihyy_loc = [61.8474, 24.2948]
nlhor_loc = [52.24035, 5.071301]
ruche_loc = [68.61304, 161.34143]
aurfi_loc = [-32.5061, 116.9668]

#%% plot all profiles
#lons = [uspfa_loc[1], uswpt_loc[1], usorv_loc[1], usbes_loc[1], usivo_loc[1], desfn_loc[1], \
#        fihyy_loc[1], nlhor_loc[1], ruche_loc[1], aurfi_loc[1]]
lons = [usmyb_loc[1], ustwt_loc[1], desfn_loc[1], \
        nlhor_loc[1]]
#lats = [uspfa_loc[0], uswpt_loc[0], usorv_loc[0], usbes_loc[0], usivo_loc[0], desfn_loc[0], \
#        fihyy_loc[0], nlhor_loc[0], ruche_loc[0], aurfi_loc[0]]
lats = [usmyb_loc[0], ustwt_loc[0], desfn_loc[0], \
        nlhor_loc[0]]
fig = plt.figure(figsize=(24,12))
ax = fig.add_axes([0.05,0.05,0.9,0.9])
m = Basemap(llcrnrlon=-180,llcrnrlat=-60,urcrnrlon=180,urcrnrlat=80,resolution='l',projection='mill',lon_0=0,lat_0=0)
lon, lat = np.meshgrid(lons, lats)
X, Y = m(lon,lat)
m.drawcoastlines(linewidth=0.25)
m.drawcountries(linewidth=0.25)
m.drawmapboundary(fill_color='#99ffff')
m.fillcontinents(color='grey',lake_color='#99ffff',zorder=0)
m.scatter(lons,lats,2000,marker='^',color='r',alpha=0.7,latlon=True)
# draw parallels.
parallels = np.arange(-90.,90.,30.)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(0.,360.,45.)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
# put text
xy = np.column_stack((X[0,:],Y[:,0]))
#xy1 = xy[0:2,:]
#xy2 = xy[2,:]
#xy3 = xy[3:10,:]
#text1 = ['US_PFa', 'US_WPT']
#text2 = ['US_ORv']
#text3 = ['US_Bes','US_IVO', 'DE_SfN', 'FI_Hyy', 'NL_Hor', 'RU_Ch2', 'AU_RFi']

xy1 = xy[0:2,:]
xy2 = xy[2,:]
xy3 = xy[3:10,:]
text1 = [' ', ' ']
text2 = [' ']
text3 = [' ', ' ']


for label, (x,y) in zip(text1, xy1):
    #ax.annotate(label, (x,y), xytext=(5, 5), textcoords='offset points')
    ax.text(x, y, label, ha='left', size=30)
ax.text(xy2[0], xy2[1], text2[0], ha='right', size=30)
for label, (x,y) in zip(text3, xy3):
    #ax.annotate(label, (x,y), xytext=(5, 5), textcoords='offset points')
    ax.text(x, y, label, ha='left', size=30)
    
ax.set_title('CH4 flux EC measurement sites')
plt.show()
#fig.savefig('./methane_sitemap.png')
    

loc = isam.latlon_2_idx(uspfa_loc[0], uspfa_loc[1])
uspfa_isam = ch4_isam[loc[0]-1, loc[1]-1]*1e9/(12*3600*365)
loc = isam.latlon_2_idx(uswpt_loc[0], uswpt_loc[1])
uswpt_isam = ch4_isam[loc[0]-1, loc[1]-1]*1e9/(12*3600*365)
loc = isam.latlon_2_idx(usorv_loc[0], usorv_loc[1])
usorv_isam = ch4_isam[loc[0]-1, loc[1]-1]*1e9/(12*3600*365)
loc = isam.latlon_2_idx(usbes_loc[0], usbes_loc[1])
usbes_isam = ch4_isam[loc[0]-1, loc[1]-1]*1e9/(12*3600*365)
loc = isam.latlon_2_idx(usivo_loc[0], usivo_loc[1])
usivo_isam = ch4_isam[loc[0]-1, loc[1]-1]*1e9/(12*3600*365)
loc = isam.latlon_2_idx(desfn_loc[0], desfn_loc[1])
desfn_isam = ch4_isam[loc[0]-1, loc[1]-1]*1e9/(12*3600*365)
loc = isam.latlon_2_idx(fihyy_loc[0], fihyy_loc[1])
fihyy_isam = ch4_isam[loc[0]-1, loc[1]-1]*1e9/(12*3600*365)
loc = isam.latlon_2_idx(nlhor_loc[0], nlhor_loc[1])
nlhor_isam = ch4_isam[loc[0]-1, loc[1]-1]*1e9/(12*3600*365)
loc = isam.latlon_2_idx(ruche_loc[0], ruche_loc[1])
ruche_isam = ch4_isam[loc[0]-1, loc[1]-1]*1e9/(12*3600*365)

In [ ]:
xy

In [ ]:
all_profid = data.index.unique()
lons = prep.getvarxls(data,'Lon',all_profid[0:16],0)
lats = prep.getvarxls(data,'Lat',all_profid[0:16],0)

# Read in model output
d14cm = pd.read_table('isam_dc14.dat', header=None, delimiter=r"\s+")
# Read in another case from the ISAM model output
# d14cm2 = pd.read_table('isam_dc14.dat', header=None, delimiter=r"\s+")
d14cm.columns = ['ID', 'Layer1', 'Layer2', 'Layer3', 'Layer4', 'Layer5', 'Layer6', 'Layer7', 'Layer8', 'Layer9', 'Layer10']
d14cm = d14cm.set_index('ID')
mod_profid = d14cm.index
# d14cm2.columns = ['ID', 'Layer1', 'Layer2', 'Layer3', 'Layer4', 'Layer5', 'Layer6', 'Layer7', 'Layer8', 'Layer9', 'Layer10']
# d14cm2 = d14cm2.set_index('ID')
# mod2_profid = d14cm2.index
z, dz, zsoih = isam.get_isam_soildp(10)

# Extract corresponding measurements for each site and make figure
data.nodedepth = data.Layer_top + (data.Layer_bottom - data.Layer_top)/2.
myarray = d14cm.index.unique()
for i in myarray:
    d14co = data.D14C_BulkLayer.loc[i]
    nd = data.nodedepth.loc[i]
    if(d14co.__class__.__name__ == 'float64'):
        Xobs = d14co   # SOC profile kgCm-3
        Yobs = nd      # 1cm to 200cm
    else:
        Xobs = d14co.as_matrix()   # SOC profile kgCm-3
        Yobs = nd.as_matrix()     # 1cm to 200cm
    Xmod = d14cm.loc[i].as_matrix()   # SOC profile kgCm-3
    Ymod = z*100.     # 1cm to 200cm
    # Xmod2 = d14cm2.loc[i].as_matrix()   # SOC profile kgCm-3
    # Ymod2 = z*100.     # 1cm to 200cm
    if (data.Site[i].__class__.__name__ == 'Series'):
        tit = data.Site[i].unique().astype('string')[0]
    else:
        tit = data.Site[i].encode('ascii','ignore')
    path = './Figs_obsvsmod_calibrate/'+str(i)+'_'+tit+'.png'
    xticks = (-1000, -800, -600, -400, -200, 0, 200)
    #status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path, xticks)
    status = socplt.plot_obsvsmod(Xobs, Yobs, Xmod, Ymod, tit, path, None, None, xticks)